In [8]:
import requests
import re
import os
from dotenv import load_dotenv
from pathlib import Path
from bs4 import BeautifulSoup

load env

In [9]:
dotenv_path = Path('../config/.env')
config = load_dotenv(dotenv_path=dotenv_path)

In [10]:
username = os.environ.get("username")
password = os.environ.get("password")

In [ ]:
import requests
from bs4 import BeautifulSoup
session = requests.Session()

login_url = 'https://bigdata.com/login'
credentials = {
    'username': username,
    'password': password
}

session.post(login_url, data=credentials)

response = session.get('https://bigdata.com/data')
soup = BeautifulSoup(response.text, 'html.parser')
print(soup.prettify())



# using beautiful soup without login (fail)

In [ ]:
import requests
import re
from bs4 import BeautifulSoup

In [ ]:
bigdata = requests.get('https://www-scopus-com.chula.idm.oclc.org/results/results.uri?sort=plf-f&src=s&sid=6a1c101025217859335e8fbcbc2a746f&sot=a&sdt=a&sl=42&s=ALL%28mahidol+university%29+AND+PUBYEAR+%3D+2024&origin=searchhistory&txGid=96d2c483c5068b1d5adbd38ade578008&sessionSearchId=6a1c101025217859335e8fbcbc2a746f&limit=10')

In [ ]:
bigdata

In [ ]:
bigdata.text

In [ ]:
soup = BeautifulSoup(bigdata.text, "lxml")
print(soup.prettify())

# Beatiful soup with login (fail due to security and privacy reason)

In [ ]:
import requests
from bs4 import BeautifulSoup

# Start a session
session = requests.Session()

# URL for the login page
login_url = "https://login.chula.idm.oclc.org/login?qurl=https://www.scopus.com%2fresults%2fresults.uri%3fsort%3dplf-f%26src%3ds%26sid%3d6a1c101025217859335e8fbcbc2a746f%26sot%3da%26sdt%3da%26sl%3d42%26s%3dALL%2528mahidol%2buniversity%2529%2bAND%2bPUBYEAR%2b%253D%2b2024%26origin%3dsearchhistory%26txGid%3d96d2c483c5068b1d5adbd38ade578008%26sessionSearchId%3d6a1c101025217859335e8fbcbc2a746f%26limit%3d10"

response = session.get(login_url)
soup = BeautifulSoup(response.text, 'html.parser')
login_action_url = "https://login.chula.idm.oclc.org" + soup.find('form', {'action': '/login'}).get('action')
hidden_inputs = soup.find_all('input', type='hidden')
form_data = {inp.get('name'): inp.get('value') for inp in hidden_inputs}

# credentila
form_data['user'] = username
form_data['pass'] = password


response = session.post(login_action_url, data=form_data)
scopus_url = "https://www-scopus-com.chula.idm.oclc.org/results/results.uri?sort=plf-f&src=s&sid=6a1c101025217859335e8fbcbc2a746f&sot=a&sdt=a&sl=42&s=ALL%28mahidol+university%29+AND+PUBYEAR+%3D+2024&origin=searchhistory&txGid=96d2c483c5068b1d5adbd38ade578008&sessionSearchId=6a1c101025217859335e8fbcbc2a746f&limit=10"
response = session.get(scopus_url)

print(response.text)



# selenium

## v.1 (only login)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Set up the Chrome driver
driver = webdriver.Chrome()

# Open the login page
driver.get('https://login.chula.idm.oclc.org/login?qurl=https://www.scopus.com')

# Wait for the input fields to be loaded
username_input = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.NAME, "user"))
)
password_input = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.NAME, "pass"))
)

# credentials
username_input.send_keys(username)
password_input.send_keys(password)

# Submit the form
password_input.submit() 

# wait til login is successful
WebDriverWait(driver, 20).until(
    EC.url_contains("scopus.com")
)

current_url = driver.current_url
print("Current URL after attempting to log in:", current_url)

if "scopus.com" in current_url:
    print("Login successful!")
else:
    print("Login failed. Still on login page.")
input("Press Enter to close the browser...")
driver.quit()



## v.2 (only first page)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time 

def main():
    # Set up the Chrome driver
    driver = webdriver.Chrome()

    try:
        # Open the login page
        driver.get('https://login.chula.idm.oclc.org/login?qurl=https://www.scopus.com')

        # Wait for the input fields to be loaded
        username_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, "user"))
        )
        password_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, "pass"))
        )

        # credentials
        username_input.send_keys(username) 
        password_input.send_keys(password)  

        # Submit the form
        password_input.submit()  

        # ensure everything is ok
        time.sleep(30)  

        # Directly navigate to the specific results page
        results_url = "https://www-scopus-com.chula.idm.oclc.org/results/results.uri?sort=plf-f&src=s&sid=6a1c101025217859335e8fbcbc2a746f&sot=a&sdt=a&sl=42&s=ALL%28mahidol+university%29+AND+PUBYEAR+%3D+2024&origin=searchhistory&txGid=96d2c483c5068b1d5adbd38ade578008&sessionSearchId=6a1c101025217859335e8fbcbc2a746f&limit=10"
        driver.get(results_url)

        print("Navigated to the specific search results page.")

        # Identify all document entries
        document_entries = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".TableItems-module__m0Z0b.TableItems-module__A6xTk"))
        )

        count = 0 
        docs = []
        # Loop through each document 
        for document in document_entries:
            texts = []
            data_elements = document.find_elements(By.CSS_SELECTOR, "td")
            for element in data_elements:
                #print(element.text)  # Prints out the text in each 'td' element
                texts.append(element.text)
            count += 1
            docs.append(texts)
            print(f"finish document {count}")
        print(docs)

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Wait for user input to close the browser
        input("Press Enter to close the browser...")
        driver.quit()

if __name__ == "__main__":
    main()


## v.3 all pages

In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time 
import csv  

def main():
    # Set up the Chrome driver
    driver = webdriver.Chrome()

    try:
        # Open the login page
        driver.get('https://login.chula.idm.oclc.org/login?qurl=https://www.scopus.com')

        # Wait for the input fields to be loaded
        username_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, "user"))
        )
        password_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, "pass"))
        )

        # credentials
        username_input.send_keys(username) 
        password_input.send_keys(password) 

        # Submit the form
        password_input.submit()  

        # ensure everything is completed
        time.sleep(30)  

        # Directly navigate to the specific results page
        results_url = "https://www-scopus-com.chula.idm.oclc.org/results/results.uri?sort=plf-f&src=s&sid=6a1c101025217859335e8fbcbc2a746f&sot=a&sdt=a&sl=42&s=ALL%28mahidol+university%29+AND+PUBYEAR+%3D+2024&origin=searchhistory&txGid=96d2c483c5068b1d5adbd38ade578008&sessionSearchId=6a1c101025217859335e8fbcbc2a746f&limit=10"
        driver.get(results_url)

        all_data = []

        # Start pagination loop
        while True:
            print("Navigated to the search results page.")
            document_entries = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".TableItems-module__m0Z0b.TableItems-module__A6xTk"))
            )

            # Loop through each document
            for document in document_entries:
                texts = [element.text for element in document.find_elements(By.CSS_SELECTOR, "td")]
                all_data.append(texts)
                #print(f"Document collected: {texts}")

            # Find the 'Next' page button and click if not disabled
            page_items = driver.find_elements(By.CSS_SELECTOR, ".page-item")
            if page_items:
                next_button = page_items[-1].find_element(By.CSS_SELECTOR, ".Button-module__nc6_8.Button-module__rphhF.Button-module__VBKvn.Button-module__MlsfC.Button-module__hK_LA.Button-module__qDdAl")
                if "disabled" not in next_button.get_attribute("class"):
                    next_button.click()
                    time.sleep(5)  # Wait for the next page to load
                else:
                    print("No more pages or 'Next' button is disabled.")
                    break
            else:
                print("No more pages or 'Next' button is not found.")
                break

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Save all data to a CSV file
        with open('scopus_data_mahidol2024.csv', 'w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            
            for row in all_data:
                writer.writerow(row)
        print("All data collected and saved to scopus_data.csv.")
        input("Press Enter to close the browser...")
        driver.quit()

if __name__ == "__main__":
    main()


Navigated to the search results page.
Navigated to the search results page.
Navigated to the search results page.
All data collected and saved to scopus_data.csv.


# connect to mongodb

In [6]:
import pandas as pd
df = pd.read_csv('../../data/modified_data_mahidol2024.csv')
df

,article
0,Correction to: Effects of customized insoles w...
1,Albumin levels in malaria patients: a systemat...
2,"Enhancing effect of natural adjuvant, pandurat..."
3,Sesamolin serves as an MYH14 inhibitor to sens...
4,Author Correction: Residual renal volume as a ...
...,...
1994,Exploring the possibility of carbon neutral di...
1995,The Effects of a School-Based Mental Health Pr...
1996,Television Viewing and Divorce in Indonesia: E...
1997,Networked Governance in Transnational Cooperat...


In [14]:
import pandas as pd
from pymongo import MongoClient

data = df.to_dict(orient='records')
mongo_uri = os.environ.get("mongo_uri")
client = MongoClient(mongo_uri)

# Select the database and collection
db = client['scrape_data']
collection = db['titles']

# Insert the data into the collection
collection.insert_many(data)

print("Data has been uploaded to MongoDB successfully.")

Data has been uploaded to MongoDB successfully.
